<a href="https://colab.research.google.com/github/jianingni/test_norani/blob/main/Fama_French_3_Factors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fama-French Three Factors


### 1.1 Center for Research in Security Prices (CRSP) Data

In [ ]:
### Connect Google Drive ###

from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
#### Import and Install Tools ###
!pip install pandasql
import os
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as sm
import numpy as np
import matplotlib.pyplot as plt
from sqlite3 import connect
from scipy.stats.mstats import winsorize, ks_2samp
import seaborn as sns
import pandasql as ps



In [ ]:
conn = connect(':memory:', timeout=10) # connect to convert between pd dataframe and sql
filePath = "/content/gdrive/My Drive/Colab Notebooks"


In [ ]:
### Import CRSP CSV ###

file = filePath + "/CRSPMonthly1990Through2022.csv"
crsp1990To2022 = pd.read_csv(file)
print(crsp1990To2022)
print(crsp1990To2022.columns)

<ipython-input-8-76ccc1132248>:4: DtypeWarning: Columns (5,9,18,19,49,58) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp1990To2022 = pd.read_csv(file)


         PERMNO      date  NAMEENDT  SHRCD  EXCHCD   SICCD    NCUSIP TICKER  \
0         10001  19900131       NaN   11.0     3.0    4920  39040610   GFGC   
1         10001  19900228       NaN   11.0     3.0    4920  39040610   GFGC   
2         10001  19900330       NaN   11.0     3.0    4920  39040610   GFGC   
3         10001  19900430       NaN   11.0     3.0    4920  39040610   GFGC   
4         10001  19900531       NaN   11.0     3.0    4920  39040610   GFGC   
...         ...       ...       ...    ...     ...     ...       ...    ...   
2975236   93436  20211130       NaN   11.0     3.0  9999.0  88160R10   TSLA   
2975237   93436  20211231       NaN   11.0     3.0  9999.0  88160R10   TSLA   
2975238   93436  20220131       NaN   11.0     3.0  9999.0  88160R10   TSLA   
2975239   93436  20220228       NaN   11.0     3.0  9999.0  88160R10   TSLA   
2975240   93436  20220331       NaN   11.0     3.0  9999.0  88160R10   TSLA   

                     COMNAM SHRCLS  ... CFACSHR    

In [ ]:
### Clean CRSP Dataframe ###

# Parse relevant variables
crsp1990To2022.to_sql("crsp1990To2022", conn, if_exists='replace')
query = '''
    SELECT DISTINCT PERMNO as permno, date, RET as ret, SHROUT as shrout, ALTPRC as altprc, EXCHCD as exchcd,
    SHRCD as shrcd, SICCD as siccd, DLRET as dlret, DLSTCD as dlstcd
    FROM crsp1990To2022
'''
crsp1990To2022 = pd.read_sql(query, conn)

In [ ]:
# Convert each column to correct variable types
crsp1990To2022['permno'] = pd.to_numeric(crsp1990To2022['permno'], errors='coerce') # security identifier
crsp1990To2022['date'] = pd.to_datetime(crsp1990To2022['date'], format='%Y%m%d') # month identifier
crsp1990To2022['ret'] = pd.to_numeric(crsp1990To2022['ret'], errors='coerce') # return
crsp1990To2022['shrout'] = pd.to_numeric(crsp1990To2022['shrout'], errors='coerce') # share outstanding (thousands)
crsp1990To2022['altprc'] = pd.to_numeric(crsp1990To2022['altprc'], errors='coerce') # last traded price in a month
crsp1990To2022['exchcd'] = pd.to_numeric(crsp1990To2022['exchcd'], errors='coerce') # exchange code
crsp1990To2022['shrcd'] = pd.to_numeric(crsp1990To2022['shrcd'], errors='coerce') # share code
crsp1990To2022['siccd'] = pd.to_numeric(crsp1990To2022['siccd'], errors='coerce') # industry code
crsp1990To2022['dlret'] = pd.to_numeric(crsp1990To2022['dlret'], errors='coerce') # delisting return
crsp1990To2022['dlstcd'] = pd.to_numeric(crsp1990To2022['dlstcd'], errors='coerce') # delisting code

# Convert returns to percent
crsp1990To2022['ret'] *= 100
crsp1990To2022['dlret'] *= 100

In [ ]:
# Only keep data after 2018
crsp2018To2022 = crsp1990To2022[crsp1990To2022['date'] >= "2018-01-01 00:00:00"]

# Only keep US-based common stocks
crsp2018To2022 = crsp2018To2022[(crsp2018To2022['shrcd'] == 10) | (crsp2018To2022['shrcd'] == 11)]

print(crsp2018To2022)


         permno       date      ret     shrout      altprc  exchcd  shrcd  \
2358      10026 2018-01-31  -8.8191    18678.0   138.44000     3.0   11.0   
2359      10026 2018-02-28  -2.9688    18678.0   134.33000     3.0   11.0   
2360      10026 2018-03-29   1.9951    18697.0   136.56000     3.0   11.0   
2361      10026 2018-04-30   0.6224    18702.0   137.41000     3.0   11.0   
2362      10026 2018-05-31   3.0638    18702.0   141.62000     3.0   11.0   
...         ...        ...      ...        ...         ...     ...    ...   
2947235   93436 2021-11-30   2.7612  1004265.0  1144.76001     3.0   11.0   
2947236   93436 2021-12-31  -7.6855  1033534.0  1056.78003     3.0   11.0   
2947237   93436 2022-01-31 -11.3609  1033534.0   936.71997     3.0   11.0   
2947238   93436 2022-02-28  -7.0768  1033534.0   870.42999     3.0   11.0   
2947239   93436 2022-03-31  23.8009  1033534.0  1077.59998     3.0   11.0   

          siccd  dlret  dlstcd  
2358     2052.0    NaN     NaN  
2359     

In [ ]:
# Calculate market cap (millions)
crsp2018To2022['market_cap'] = abs(crsp2018To2022['shrout'] * crsp2018To2022['altprc']) / 1000
crsp2018To2022['market_cap'].replace(0, np.NaN)

,market_cap
2358,2.585782e+03
2359,2.509016e+03
2360,2.553262e+03
2361,2.569842e+03
2362,2.648577e+03
...,...
2947235,1.149642e+06
2947236,1.092218e+06
2947237,9.681319e+05
2947238,8.996190e+05


In [ ]:
# Define exchange labels and keep only relevant labels
crsp2018To2022['exchange'] = ""
for i in crsp2018To2022.index:
    if crsp2018To2022['exchcd'][i] in [1, 31]:
        crsp2018To2022['exchange'][i] = "NYSE"
    elif crsp2018To2022['exchcd'][i] in [2, 32]:
        crsp2018To2022['exchange'][i] = "AMEX"
    elif crsp2018To2022['exchcd'][i] in [3, 33]:
        crsp2018To2022['exchange'][i] = "NASDAQ"
    else:
        crsp2018To2022['exchange'][i] = "Other"
crsp2018To2022 = crsp2018To2022[crsp2018To2022['exchange'] != "Other"]

流式输出内容被截断，只能显示最后 5000 行内容。
<ipython-input-13-8698a9f18db6>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp2018To2022['exchange'][i] = "NASDAQ"
<ipython-input-13-8698a9f18db6>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp2018To2022['exchange'][i] = "NASDAQ"
<ipython-input-13-8698a9f18db6>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp2018To2022['exchange'][i] = "NASDAQ"
<ipython-input-13-8698a9f18db6>:9: SettingWith

In [ ]:
# Adjust delisting returns
crsp2018To2022['ret_adj'] = ""
for i in crsp2018To2022.index:
    if pd.isnull(crsp2018To2022['dlstcd'][i]):
        crsp2018To2022['ret_adj'][i] = crsp2018To2022['ret'][i]
    elif (pd.notnull(crsp2018To2022['dlstcd'][i])) and (pd.notnull(crsp2018To2022['dlret'][i])):
        crsp2018To2022['ret_adj'][i] = crsp2018To2022['dlret'][i]
    elif (551 <= crsp2018To2022['dlstcd'][i] <= 574) or (crsp2018To2022['dlstcd'][i] in [500, 520, 580, 584]):
        crsp2018To2022['ret_adj'][i] = -30
    else:
        crsp2018To2022['ret_adj'][i] = -100

流式输出内容被截断，只能显示最后 5000 行内容。
<ipython-input-14-70b21d04fb0d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp2018To2022['ret_adj'][i] = crsp2018To2022['ret'][i]
<ipython-input-14-70b21d04fb0d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp2018To2022['ret_adj'][i] = crsp2018To2022['ret'][i]
<ipython-input-14-70b21d04fb0d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp2018To2022['ret_adj'][i] = crsp2018To2022['ret'][i]
<

In [ ]:
# Reindex and drop unneeded columns
crsp2018To2022 = crsp2018To2022.reset_index()
crsp2018To2022.drop(['shrcd', 'dlret', 'dlstcd', 'index'], inplace=True, axis=1)

print(crsp2018To2022)
print(crsp2018To2022.columns)

        permno       date      ret     shrout      altprc  exchcd   siccd  \
0        10026 2018-01-31  -8.8191    18678.0   138.44000     3.0  2052.0   
1        10026 2018-02-28  -2.9688    18678.0   134.33000     3.0  2052.0   
2        10026 2018-03-29   1.9951    18697.0   136.56000     3.0  2052.0   
3        10026 2018-04-30   0.6224    18702.0   137.41000     3.0  2052.0   
4        10026 2018-05-31   3.0638    18702.0   141.62000     3.0  2052.0   
...        ...        ...      ...        ...         ...     ...     ...   
193370   93436 2021-11-30   2.7612  1004265.0  1144.76001     3.0  9999.0   
193371   93436 2021-12-31  -7.6855  1033534.0  1056.78003     3.0  9999.0   
193372   93436 2022-01-31 -11.3609  1033534.0   936.71997     3.0  9999.0   
193373   93436 2022-02-28  -7.0768  1033534.0   870.42999     3.0  9999.0   
193374   93436 2022-03-31  23.8009  1033534.0  1077.59998     3.0  9999.0   

          market_cap exchange  ret_adj  
0       2.585782e+03   NASDAQ  -8.

### 2.1 Compustat Merged (CCM) Data

In [ ]:
### Import CRSP/Compustat Merged CSV ###
file = filePath + "/ccmfund1990to2022.csv"
ccmFund1990To2022 = pd.read_csv(file)

print(ccmFund1990To2022)
print(ccmFund1990To2022.columns)

<ipython-input-16-d44ff9fa1d75>:3: DtypeWarning: Columns (2,17,19,24,33,37,952,953,955,956,957,962,976,977,982) have mixed types. Specify dtype option on import or set low_memory=False.
  ccmFund1990To2022 = pd.read_csv(file)


         GVKEY LINKPRIM LIID LINKTYPE  LPERMNO  LPERMCO    LINKDT LINKENDDT  \
0         1004        P    1       LU    54594    20000  19720424         E   
1         1004        P    1       LU    54594    20000  19720424         E   
2         1004        P    1       LU    54594    20000  19720424         E   
3         1004        P    1       LU    54594    20000  19720424         E   
4         1004        P    1       LU    54594    20000  19720424         E   
...        ...      ...  ...      ...      ...      ...       ...       ...   
209121  345980        P    1       LC    20333    57666  20201216         E   
209122  345980        P    1       LC    20333    57666  20201216         E   
209123  347007        P    1       LC    15533    55364  20210310         E   
209124  347085        P    1       LC    21069    58208  20210401         E   
209125  349530        P    1       LC    17324    56274  20210709         E   

        datadate   fyear  ... priusa     sic spcind

In [ ]:
### Clean CRSP/Compustat Merged Dataframe ###

# Parse relevant variables
ccmFund1990To2022.to_sql("ccmFund1990To2022", conn, if_exists='replace')
query = '''
    SELECT DISTINCT GVKEY, LPERMNO, datadate, LINKTYPE, LINKENDDT, seq, ceq, at, lt, txditc, txdb, itcb, pstkrv, pstkl, pstk, indfmt, datafmt
    FROM ccmFund1990To2022
'''
compustat1990To2022 = pd.read_sql(query, conn)

In [ ]:
# Convert each column to correct variable type
compustat1990To2022['GVKEY'] = pd.to_numeric(compustat1990To2022['GVKEY'], errors='coerce') # firm identifier
compustat1990To2022['LPERMNO'] = pd.to_numeric(compustat1990To2022['LPERMNO'], errors='coerce') # stock identifier
compustat1990To2022['datadate'] = pd.to_datetime(compustat1990To2022['datadate'], format='%Y%m%d') # date of report
compustat1990To2022['LINKTYPE'] = compustat1990To2022['LINKTYPE'].apply(str) # link type
compustat1990To2022['LINKENDDT'] = pd.to_datetime(compustat1990To2022['LINKENDDT'], format='%Y%m%d', errors='coerce') # link's date of expiration
compustat1990To2022['seq'] = pd.to_numeric(compustat1990To2022['seq'], errors='coerce') # stockholder's equity
compustat1990To2022['ceq'] = pd.to_numeric(compustat1990To2022['ceq'], errors='coerce') # total equity
compustat1990To2022['at'] = pd.to_numeric(compustat1990To2022['at'], errors='coerce') # total assets
compustat1990To2022['lt'] = pd.to_numeric(compustat1990To2022['lt'], errors='coerce') # total liabilities
compustat1990To2022['txditc'] = pd.to_numeric(compustat1990To2022['txditc'], errors='coerce') # deferred taxes and investment tax credit
compustat1990To2022['txdb'] = pd.to_numeric(compustat1990To2022['txdb'], errors='coerce') # deferred taxes
compustat1990To2022['itcb'] = pd.to_numeric(compustat1990To2022['itcb'], errors='coerce') # investment tax credit
compustat1990To2022['pstkrv'] = pd.to_numeric(compustat1990To2022['pstkrv'], errors='coerce') # preferred stock redemption value
compustat1990To2022['pstkl'] = pd.to_numeric(compustat1990To2022['pstkl'], errors='coerce') # preferred stock liquidating value
compustat1990To2022['pstk'] = pd.to_numeric(compustat1990To2022['pstk'], errors='coerce') # preferred stock par value
compustat1990To2022['indfmt'] = compustat1990To2022['indfmt'].apply(str) # industry format
compustat1990To2022['datafmt'] = compustat1990To2022['datafmt'].apply(str) # data format


In [ ]:
# Only keep dates after 2018
compustat2018To2022 = compustat1990To2022[compustat1990To2022['datadate'] >= "2018-01-01 00:00:00"]

# Only keep correct formats
compustat2018To2022 = compustat2018To2022[compustat2018To2022['indfmt'] == "INDL"]
compustat2018To2022 = compustat2018To2022[compustat2018To2022['datafmt'] == "STD"]

# Only keep valid links
compustat2018To2022 = compustat2018To2022[(compustat2018To2022['LINKTYPE'] == "LU") | (compustat2018To2022['LINKTYPE'] == "LC")]

# Only keep links active at datadate
compustat2018To2022 = compustat2018To2022[(compustat2018To2022['datadate'] <= compustat2018To2022['LINKENDDT']) | pd.isnull(compustat2018To2022['LINKENDDT'])]


In [ ]:
# Calculate book value
compustat2018To2022.to_sql("compustat2018To2022", conn, if_exists='replace')
query = '''
    SELECT DISTINCT GVKEY as gvkey, LPERMNO as permno, datadate,
    COALESCE(seq, ceq + pstk, at - lt) + COALESCE(txditc, txdb + itcb, 0) - COALESCE(pstkrv, pstkl, pstk, 0) as book_value
    FROM compustat2018To2022
'''
compustat2018To2022 = pd.read_sql(query, conn)
compustat2018To2022['book_value'] = pd.to_numeric(compustat2018To2022['book_value'], errors='coerce')

In [ ]:
# Only keep the last observation of each year
compustat2018To2022['datadate'] = pd.to_datetime(compustat2018To2022['datadate'])
compustat2018To2022['year'] = compustat2018To2022['datadate'].dt.year
compustat2018To2022 = compustat2018To2022.sort_values(by=['permno', 'datadate'], ascending=True)
compustat2018To2022 = compustat2018To2022.reset_index()

toDrop = []
for i in range(len(compustat2018To2022) - 1):
    if (compustat2018To2022['permno'][i] == compustat2018To2022['permno'][i + 1]) and (compustat2018To2022['year'][i] == compustat2018To2022['year'][i + 1]):
        toDrop.append(i)

compustat2018To2022.drop(toDrop, inplace=True, axis=0)
compustat2018To2022.drop('index', inplace=True, axis=1)

In [ ]:
# Drop missing/infinite values
compustat2018To2022.replace([np.inf, -np.inf], np.nan, inplace=True)
compustat2018To2022 = compustat2018To2022.dropna()

# Reindex
compustat2018To2022 = compustat2018To2022.reset_index()
compustat2018To2022.drop('index', inplace=True, axis=1)


In [ ]:
# Drop missing/infinite values
compustat2018To2022.replace([np.inf, -np.inf], np.nan, inplace=True)
compustat2018To2022 = compustat2018To2022.dropna()

# Reindex
compustat2018To2022 = compustat2018To2022.reset_index()
compustat2018To2022.drop('index', inplace=True, axis=1)


In [ ]:
# Add reference date for matching (6/1 of next calendar year)
compustat2018To2022['year'] = pd.to_numeric(compustat2018To2022['year'], errors='coerce')
compustat2018To2022['reference_date'] = compustat2018To2022['year'] + 1
compustat2018To2022['reference_date'] = compustat2018To2022['reference_date'].apply(str)
compustat2018To2022['reference_date'] += "-06-01"
compustat2018To2022['reference_date'] = pd.to_datetime(compustat2018To2022['reference_date'])
compustat2018To2022.drop('year', inplace=True, axis=1)

print(compustat2018To2022)
print(compustat2018To2022.columns)

        gvkey  permno   datadate  book_value reference_date
0       12825   10026 2018-09-30     811.413     2019-06-01
1       12825   10026 2019-09-30     895.671     2020-06-01
2       12825   10026 2020-09-30     873.911     2021-06-01
3       12825   10026 2021-09-30     907.232     2022-06-01
4       12096   10028 2018-12-31       8.413     2019-06-01
...       ...     ...        ...         ...            ...
20524  184259   93434 2021-06-30      74.602     2022-06-01
20525  184996   93436 2018-12-31    4923.243     2019-06-01
20526  184996   93436 2019-12-31    6618.000     2020-06-01
20527  184996   93436 2020-12-31   22376.000     2021-06-01
20528  184996   93436 2021-12-31   30213.000     2022-06-01

[20529 rows x 5 columns]
Index(['gvkey', 'permno', 'datadate', 'book_value', 'reference_date'], dtype='object')


### 3.1 Construct Stocks Data

In [ ]:
# Parse relevant variables from CRSP
crsp2018To2022.to_sql("crsp2018To2022", conn, if_exists='replace')

# Corrected SQL query
query = '''
    SELECT DISTINCT permno, date, exchange, ret_adj as ret, market_cap
    FROM crsp2018To2022
'''

# Execute the query and load into DataFrame
stocks2018To2022 = pd.read_sql(query, conn)

# Drop missing/infinite values
stocks2018To2022.replace([np.inf, -np.inf], np.nan, inplace=True)
stocks2018To2022 = stocks2018To2022.dropna()


In [ ]:
# Add reference date for each stock (last 6/1)
stocks2018To2022['date'] = pd.to_datetime(stocks2018To2022['date'])
stocks2018To2022['reference_date'] = ""

for i in stocks2018To2022.index:
    if stocks2018To2022['date'][i].month < 6:
        stocks2018To2022['reference_date'][i] = str(stocks2018To2022['date'][i].year - 1) + "-06-01"
    else:
        stocks2018To2022['reference_date'][i] = str(stocks2018To2022['date'][i].year) + "-06-01"

stocks2018To2022['reference_date'] = pd.to_datetime(stocks2018To2022['reference_date'])


流式输出内容被截断，只能显示最后 5000 行内容。
<ipython-input-30-0b65b050ea15>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks2018To2022['reference_date'][i] = str(stocks2018To2022['date'][i].year) + "-06-01"
<ipython-input-30-0b65b050ea15>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks2018To2022['reference_date'][i] = str(stocks2018To2022['date'][i].year) + "-06-01"
<ipython-input-30-0b65b050ea15>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [ ]:
# Left join stocks dataframe with compustat dataframe
stocks2018To2022.to_sql("stocks2018To2022", conn, if_exists = 'replace')
compustat2018To2022.to_sql("compustat2018To2022", conn, if_exists = 'replace')

query = '''
    SELECT DISTINCT stocks2018To2022.*, compustat2018To2022.gvkey, compustat2018To2022.datadate,
    compustat2018To2022.book_value
    FROM stocks2018To2022
    LEFT JOIN compustat2018To2022
    ON stocks2018To2022.permno = compustat2018To2022.permno
    AND stocks2018To2022.reference_date = compustat2018To2022.reference_date
'''

stocks2018To2022 = pd.read_sql(query, conn)
stocks2018To2022.drop('index', inplace = True, axis = 1)


In [ ]:
# Add market equity data from end of year y-1 (used for BM ratio of reference date (year y))
stocks2018To2022['date'] = pd.to_datetime(stocks2018To2022['date'])
stocksMarketEquity = stocks2018To2022[stocks2018To2022['date'].dt.month == 12]
stocksMarketEquity['reference_date'] = stocksMarketEquity['date'].dt.year + 1
stocksMarketEquity['reference_date'] = stocksMarketEquity['reference_date'].apply(str)
stocksMarketEquity['reference_date'] += "-06-01"
stocksMarketEquity['reference_date'] = pd.to_datetime(stocksMarketEquity['reference_date'])
stocksMarketEquity.to_sql("stocksMarketEquity", conn, if_exists = 'replace')

query = '''
    SELECT DISTINCT permno, reference_date, market_cap as market_equity
    FROM stocksMarketEquity
'''

stocksMarketEquity = pd.read_sql(query, conn)
stocks2018To2022.to_sql("stocks2018To2022", conn, if_exists = 'replace')
stocksMarketEquity.to_sql("stocksMarketEquity", conn, if_exists = 'replace')

query = '''
    SELECT DISTINCT stocks2018To2022.*, stocksMarketEquity.market_equity
    FROM stocks2018To2022
    LEFT JOIN stocksMarketEquity
    ON stocks2018To2022.permno = stocksMarketEquity.permno
    AND stocks2018To2022.reference_date = stocksMarketEquity.reference_date
'''

stocks2018To2022 = pd.read_sql(query, conn)
stocks2018To2022.drop('index', inplace = True, axis = 1)
stocks2018To2022 = stocks2018To2022.dropna()
stocks2018To2022 = stocks2018To2022.reset_index()
stocks2018To2022.drop('index', inplace = True, axis = 1)


<ipython-input-34-3a5224e68377>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocksMarketEquity['reference_date'] = stocksMarketEquity['date'].dt.year + 1
<ipython-input-34-3a5224e68377>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocksMarketEquity['reference_date'] = stocksMarketEquity['reference_date'].apply(str)
<ipython-input-34-3a5224e68377>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [ ]:
# Drop missing/infinite values again and only keep dates after 2020
stocks2018To2022.replace([np.inf, -np.inf], np.nan, inplace=True)
stocks2018To2022 = stocks2018To2022.dropna()

stocks2018To2022['date'] = pd.to_datetime(stocks2018To2022['date'])
stocks2018To2022 = stocks2018To2022[stocks2018To2022['date'] >= '2018-01-01 00:00:00']

print(stocks2018To2022)

        permno       date exchange       ret    market_cap  \
0        10026 2019-06-28   NASDAQ     0.373  3.030689e+03   
1        10026 2019-07-31   NASDAQ   15.4644  3.501411e+03   
2        10026 2019-08-30   NASDAQ    3.8851  3.637443e+03   
3        10026 2019-09-30   NASDAQ   -0.2901  3.627840e+03   
4        10026 2019-10-31   NASDAQ   -0.6458  3.604410e+03   
...        ...        ...      ...       ...           ...   
115662   93436 2021-11-30   NASDAQ    2.7612  1.149642e+06   
115663   93436 2021-12-31   NASDAQ   -7.6855  1.092218e+06   
115664   93436 2022-01-31   NASDAQ  -11.3609  9.681319e+05   
115665   93436 2022-02-28   NASDAQ   -7.0768  8.996190e+05   
115666   93436 2022-03-31   NASDAQ      -100  1.113736e+06   

             reference_date     gvkey             datadate  book_value  \
0       2019-06-01 00:00:00   12825.0  2018-09-30 00:00:00     811.413   
1       2019-06-01 00:00:00   12825.0  2018-09-30 00:00:00     811.413   
2       2019-06-01 00:00:00   128

### 4.1 Sort the Firm by Size

In [ ]:
### Size Sort ###

# Get NYSE stocks' sizes in June of each year
stocks2018To2022['date'] = pd.to_datetime(stocks2018To2022['date'])
sizeBreakpoints = stocks2018To2022[(stocks2018To2022['date'].dt.month == 6) & (stocks2018To2022['exchange'] == "NYSE")]
sizeBreakpoints.to_sql("sizeBreakpoints", conn, if_exists='replace')

query = '''
    SELECT DISTINCT permno, reference_date, market_cap
    FROM sizeBreakpoints
'''

sizeBreakpoints = pd.read_sql(query, conn)
sizeBreakpoints['reference_date'] = pd.to_datetime(sizeBreakpoints['reference_date'])
g = sizeBreakpoints.groupby(['reference_date'])


In [ ]:
# Create new dataframe to sort sizes
sizeSorts = stocks2018To2022[(stocks2018To2022['date'].dt.month == 6)]

# Calculate median size for each year
sizeSorts['size_median'] = ""
for i in sizeSorts.index:
    sizeSorts['size_median'][i] = g.get_group(sizeSorts['reference_date'][i]).reset_index()['market_cap'].quantile(0.5)


流式输出内容被截断，只能显示最后 5000 行内容。
<ipython-input-40-a29edef547e0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sizeSorts['size_median'][i] = g.get_group(sizeSorts['reference_date'][i]).reset_index()['market_cap'].quantile(0.5)
<ipython-input-40-a29edef547e0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sizeSorts['size_median'][i] = g.get_group(sizeSorts['reference_date'][i]).reset_index()['market_cap'].quantile(0.5)
<ipython-input-40-a29edef547e0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [ ]:
# Sort sizes into big and small portfolios
sizeSorts['size_portfolio'] = ""

for i in sizeSorts.index:
    if sizeSorts['market_cap'][i] > sizeSorts['size_median'][i]:
        sizeSorts['size_portfolio'][i] = "B"
    else:
        sizeSorts['size_portfolio'][i] = "S"


流式输出内容被截断，只能显示最后 5000 行内容。
<ipython-input-41-2158ade68a76>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sizeSorts['size_portfolio'][i] = "B"
<ipython-input-41-2158ade68a76>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sizeSorts['size_portfolio'][i] = "B"
<ipython-input-41-2158ade68a76>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sizeSorts['size_portfolio'][i] = "S"
<ipython-input-41-2158ade68a76>:8: SettingWithCopyWarning:

In [ ]:
# Left join size portfolio back into stocks dataframe
stocks2018To2022.to_sql("stocks2018To2022", conn, if_exists='replace')
sizeSorts.to_sql("sizeSorts", conn, if_exists='replace')

query = '''
    SELECT DISTINCT stocks2018To2022.*, sizeSorts.size_portfolio
    FROM stocks2018To2022
    LEFT JOIN sizeSorts
    ON stocks2018To2022.reference_date = sizeSorts.reference_date
    AND stocks2018To2022.permno = sizeSorts.permno
'''

stocks2018To2022 = pd.read_sql(query, conn)
stocks2018To2022.drop('index', inplace=True, axis=1)

print(stocks2018To2022)
print(stocks2018To2022.columns)


        permno                 date exchange       ret    market_cap  \
0        10026  2019-06-28 00:00:00   NASDAQ     0.373  3.030689e+03   
1        10026  2019-07-31 00:00:00   NASDAQ   15.4644  3.501411e+03   
2        10026  2019-08-30 00:00:00   NASDAQ    3.8851  3.637443e+03   
3        10026  2019-09-30 00:00:00   NASDAQ   -0.2901  3.627840e+03   
4        10026  2019-10-31 00:00:00   NASDAQ   -0.6458  3.604410e+03   
...        ...                  ...      ...       ...           ...   
115662   93436  2021-11-30 00:00:00   NASDAQ    2.7612  1.149642e+06   
115663   93436  2021-12-31 00:00:00   NASDAQ   -7.6855  1.092218e+06   
115664   93436  2022-01-31 00:00:00   NASDAQ  -11.3609  9.681319e+05   
115665   93436  2022-02-28 00:00:00   NASDAQ   -7.0768  8.996190e+05   
115666   93436  2022-03-31 00:00:00   NASDAQ      -100  1.113736e+06   

             reference_date     gvkey             datadate  book_value  \
0       2019-06-01 00:00:00   12825.0  2018-09-30 00:00:00   

### 5.1 Sort the Firm by Value

In [ ]:
print(valueBreakpoints.columns)

Index(['permno', 'reference_date', 'bm_ratio'], dtype='object')


In [ ]:
# Get value breakpoints of NYSE stocks
stocks2018To2022['date'] = pd.to_datetime(stocks2018To2022['date'])
valueBreakpoints = stocks2018To2022[(stocks2018To2022['date'].dt.month == 6) & (stocks2018To2022['exchange'] == "NYSE")]
valueBreakpoints.to_sql("valueBreakpoints", conn, if_exists='replace')

3620

In [ ]:
# Calculate bm_ratio
valueBreakpoints['bm_ratio'] = valueBreakpoints['market_cap']/valueBreakpoints['book_value']

# Save updated valueBreakpoints DataFrame to SQL
valueBreakpoints.to_sql("valueBreakpoints", conn, if_exists='replace')


<ipython-input-80-54cdd529a2fa>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valueBreakpoints['bm_ratio'] = valueBreakpoints['market_cap']/valueBreakpoints['book_value']


3620

In [ ]:
### Value Sort ###

query = '''
    SELECT DISTINCT permno, reference_date, bm_ratio
    FROM valueBreakpoints
'''

valueBreakpoints = pd.read_sql(query, conn)
valueBreakpoints['reference_date'] = pd.to_datetime(valueBreakpoints['reference_date'])
g = valueBreakpoints.groupby(['reference_date'])


In [ ]:
# Create new dataframe to sort values
valueSorts = stocks2018To2022[(stocks2018To2022['date'].dt.month == 6)]

# Calculate 30th and 70th quantile of book values
valueSorts['value_q30'] = ""
valueSorts['value_q70'] = ""

for i in valueSorts.index:
    valueSorts['value_q30'][i] = g.get_group(g.get_group(valueSorts['reference_date'][i]).reset_index()['reference_date'][0])['bm_ratio'].quantile(0.3)
    valueSorts['value_q70'][i] = g.get_group(g.get_group(valueSorts['reference_date'][i]).reset_index()['reference_date'][0])['bm_ratio'].quantile(0.7)


流式输出内容被截断，只能显示最后 5000 行内容。
<ipython-input-82-e2883829bbac>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valueSorts['value_q70'][i] = g.get_group(g.get_group(valueSorts['reference_date'][i]).reset_index()['reference_date'][0])['bm_ratio'].quantile(0.7)
<ipython-input-82-e2883829bbac>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valueSorts['value_q30'][i] = g.get_group(g.get_group(valueSorts['reference_date'][i]).reset_index()['reference_date'][0])['bm_ratio'].quantile(0.3)
<ipython-input-82-e2883829bbac>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cave

KeyboardInterrupt: 

In [ ]:
# Sort values into high, medium, and low portfolios
valueSorts['value_portfolio'] = ""

for i in valueSorts.index:
    if valueSorts['bm_ratio'][i] > valueSorts['value_q70'][i]:
        valueSorts['value_portfolio'][i] = "H"
    elif valueSorts['value_q30'][i] < valueSorts['bm_ratio'][i] <= valueSorts['value_q70'][i]:
        valueSorts['value_portfolio'][i] = "M"
    else:
        valueSorts['value_portfolio'][i] = "L"


<ipython-input-67-5698e426e95c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valueSorts['value_portfolio'] = ""


KeyError: 'bm_ratio'

In [ ]:
# Left join value portfolio back into stocks dataframe
stocks2018To2022.to_sql("stocks2018To2022", conn, if_exists='replace')
valueSorts.to_sql("valueSorts", conn, if_exists='replace')

query = '''
    SELECT DISTINCT stocks2018To2022.*, valueSorts.value_portfolio
    FROM stocks2018To2022
    LEFT JOIN valueSorts
    ON stocks2018To2022.reference_date = valueSorts.reference_date
    AND stocks2018To2022.permno = valueSorts.permno
'''

stocks2018To2022 = pd.read_sql(query, conn)
stocks2018To2022.drop('index', inplace=True, axis=1)

print(stocks2018To2022)
print(stocks2018To2022.columns)


        permno                 date exchange       ret    market_cap  \
0        10026  2019-06-28 00:00:00   NASDAQ     0.373  3.030689e+03   
1        10026  2019-07-31 00:00:00   NASDAQ   15.4644  3.501411e+03   
2        10026  2019-08-30 00:00:00   NASDAQ    3.8851  3.637443e+03   
3        10026  2019-09-30 00:00:00   NASDAQ   -0.2901  3.627840e+03   
4        10026  2019-10-31 00:00:00   NASDAQ   -0.6458  3.604410e+03   
...        ...                  ...      ...       ...           ...   
115662   93436  2021-11-30 00:00:00   NASDAQ    2.7612  1.149642e+06   
115663   93436  2021-12-31 00:00:00   NASDAQ   -7.6855  1.092218e+06   
115664   93436  2022-01-31 00:00:00   NASDAQ  -11.3609  9.681319e+05   
115665   93436  2022-02-28 00:00:00   NASDAQ   -7.0768  8.996190e+05   
115666   93436  2022-03-31 00:00:00   NASDAQ      -100  1.113736e+06   

             reference_date     gvkey             datadate  book_value  \
0       2019-06-01 00:00:00   12825.0  2018-09-30 00:00:00   